In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math
import datetime

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

objc[58176]: Class CaptureDelegate is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x15cece480) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x137e80860). One of the two will be used. Which one is undefined.
objc[58176]: Class CVWindow is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x15cece4d0) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1224a0a68). One of the two will be used. Which one is undefined.
objc[58176]: Class CVView is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x15cece4f8) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packa

In [3]:
# Capture video via webcam

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow("Media Pipe feed", frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
for i in range (1,5):
    cv2.waitKey(1)

### 1. Make Detections

In [ ]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("./data/db_curl_1.mov")
    # cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Media Pipe feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1,5):
        cv2.waitKey(1)

### 2. Determine joints

In [6]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("../data/db_curl_1.mov")
    # cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            print("No landmarks found")
            pass

        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Media Pipe feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1,5):
        cv2.waitKey(1)

In [18]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.55278516
y: 0.35183862
z: -0.6540659
visibility: 0.9998903

In [19]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]


x: 0.4054467
y: 0.7724296
z: -0.57371765
visibility: 0.9798264

In [24]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]


x: 0.53770936
y: 0.6176323
z: -0.6611603
visibility: 0.9961536

### 3. Calculate Angles

In [3]:
def calculate_angles(point1: list, point2: list, point3: list) -> float:
    point1 = np.array(point1)
    point2 = np.array(point2)
    point3 = np.array(point3)

    # Calculate algo
    angleInRad = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angleInDeg = np.abs(angleInRad * 180.0 / np.pi)

    angleInDeg = angleInDeg if angleInDeg <= 180 else 360 - angleInDeg
    return angleInDeg

In [7]:
shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]
elbow = [ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]
wrist = [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]

In [8]:
shoulder, elbow, wrist

([0.5346263647079468, 0.3458312451839447],
 [0.5057958960533142, 0.568242609500885],
 [0.32630857825279236, 0.5027188658714294])

In [9]:
calculate_angles(shoulder, elbow, wrist)

77.33073768507691

In [4]:
from numpy.lib.function_base import angle


VIDEO_DIM = [946, 1088]

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("../data/db_curl_1.mov")
    # cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get joints' coordinates
            shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]
            elbow = [ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]
            wrist = [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]

            angle = calculate_angles(shoulder, elbow, wrist)

            # Visualize
            cv2.putText(image, str(angle), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(e)
            pass

        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Media Pipe feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1,5):
        cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### 4. Counter

In [20]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)


def save_frame_as_image(frame, message: str = None):
    '''
    Save a frame as image
    '''
    now = datetime.datetime.now()

    if message:
        cv2.putText(frame, message, (50, 150), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 3, cv2.LINE_AA)

    cv2.imwrite(f"../data/bicep_{now}.jpg", frame)
    

In [24]:
VIDEO_DIM = [946, 1088]

# Params for counter
STAGE_UP_THRESHOLD = 90
STAGE_DOWN_THRESHOLD = 120
stage = "down"
counter = 0

# Params to catch FULL RANGE OF MOTION error
PEAK_CONTRACTION_THRESHOLD = 70
peak_contraction_degree = 1000
peaked_framed = None

# Params to catch LOOSE UPPER ARM error
LOOSE_UPPER_ARM = False

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("../data/db_curl_bad_1.mov")
    fps = cap.get(cv2.CAP_PROP_FPS)
    totalNoFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    durationInSeconds = totalNoFrames / fps

    print(f"Duration: {durationInSeconds} s, FPS: {fps}")

    while cap.isOpened():
        ret, frame = cap.read()

        frame = rescale_frame(frame, 80)
        if not ret:
            break

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get joints' coordinates
            shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y ]
            elbow = [ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y ]
            wrist = [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y ]

            angle = int(calculate_angles(shoulder, elbow, wrist))

            # Curl counter
            if angle > STAGE_DOWN_THRESHOLD:
                stage = "down"
            elif angle < STAGE_UP_THRESHOLD and stage == "down":
                stage = "up"
                counter += 1
            
            # Calculate the angle between the upper arm (shoulder & joint) and the Y axis
            shoulder_projection = [ shoulder[0], 1 ] # Represent the projection of the shoulder to the X axis
            ground_upper_arm_angle = int(calculate_angles(elbow, shoulder, shoulder_projection))

            # Visualize
            cv2.putText(image, str(angle), tuple(np.multiply(elbow, VIDEO_DIM).astype(int)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(ground_upper_arm_angle), tuple(np.multiply(shoulder, VIDEO_DIM).astype(int)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # EVALUATE FOR CORRECTIONS
            if ground_upper_arm_angle > 40:
                # Limit the saved frame
                if not LOOSE_UPPER_ARM:
                    LOOSE_UPPER_ARM = True
                    save_frame_as_image(image, "Loose upper arm")
                else:
                    pass
            else:
                LOOSE_UPPER_ARM = False

            # Evaluate peak contraction
            if stage == "up" and angle < peak_contraction_degree:
                # Capture peaked contraction every rep
                peak_contraction_degree = angle
                peaked_framed = image
                
            elif stage == "down":
                # Evaluate if the peak is higher than the threshold than capture that image and marked as an error
                if peak_contraction_degree != 1000 and peak_contraction_degree >= PEAK_CONTRACTION_THRESHOLD:
                    print(peak_contraction_degree)
                    save_frame_as_image(peaked_framed, "Peak contraction could be better")
                
                # Reset params
                peak_contraction_degree = 1000
                peaked_framed = None

        except Exception as e:
            print(e)
            pass

        # Visualize Counter
        cv2.putText(image, stage + " " + str(counter), (50, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Media Pipe feed", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1, 5):
        cv2.waitKey(1)

Duration: 44.35 s, FPS: 60.0
